# Creating dataset from LIDC CT scans

With this notebook volumes around the tumors and associated annotations are extracted. <br/>
Extracted annotations include malignancy status and biomarkers such as diameter, spiculation, calcification ..

In [1]:
import pandas as pd
import numpy as np
import glob
import pylidc as pl
from pylidc.utils import consensus
import torchio as tio
import matplotlib.pyplot as plt
import torch
from scipy import ndimage
import pickle
import sys, os
import ctypes
from contextlib import contextmanager
from pathlib import Path
from tqdm import tqdm
import pickle
from torchvision import transforms

### define paths and parameters

In [2]:
# define path to LIDC-IDRI data
base_path = "/home/lbrocki/LIDC/LIDC-IDRI"
# define path to save the dataset
save_path = "/home/lbrocki/ConRad/dataset"
# define side length of tumor volume in mm
h = 32

### helper code to catch warning from torchio

In [3]:
# this code is only needed to catch the warning from torchio, samples with a warning are excluded from the dataset
def flush(stream):
    try:
        ctypes.libc.fflush(None)
        stream.flush()
    except (AttributeError, ValueError, IOError):
        pass  # unsupported
def fileno(file_or_fd):
    fd = getattr(file_or_fd, "fileno", lambda: file_or_fd)()
    if not isinstance(fd, int):
        raise ValueError("Expected a file (`.fileno()`) or a file descriptor")
    return fd
@contextmanager
def stdout_redirected(to=os.devnull, stdout=None):
    if stdout is None:
        stdout = sys.stdout
    stdout_fd = fileno(stdout)

    with os.fdopen(os.dup(stdout_fd), "wb") as copied:
        flush(stdout)
        try:
            os.dup2(fileno(to), stdout_fd)  # $ exec >&to
        except ValueError:  # filename
            with open(to, "wb") as to_file:
                os.dup2(to_file.fileno(), stdout_fd)  # $ exec > to
        try:
            yield stdout  # allow code to be run with the redirected stdout
        finally:
            flush(stdout)
            os.dup2(copied.fileno(), stdout_fd)  # $ exec >&copied

### define the extraction of nodule volumes

In [4]:
# extract slices of dimension h x h x h around nodule center 
# and the corresponding segmentations
def process_nodule_vol(nodule, h=32):  
    dicom = nodule[0].scan.get_path_to_dicom_files()
    median_malig = np.median([nod.malignancy for nod in nodule])
    
    
#     # catch errors from torchio and throw an exception so that this nodule is skipped
    with open("output.txt", "w") as f, stdout_redirected(f, stdout=sys.stderr):
        tio_image = tio.ScalarImage(dicom)
        spacing = tio_image.spacing
        # resample isotropically to 1mm spacing
        transform = tio.Resample(1)
        res_image = transform(tio_image)
        res_data = torch.movedim(res_image.data, (0,1,2,3), (0,2,1,3))
        
    with open("output.txt") as f:
        content = f.read()
    if "warning" in content.lower():
        raise RuntimeError("SimpleITK Warning .. skip!")
    open("output.txt", "w").close()
    
    cmask,cbbox,masks = consensus(nodule, clevel=0.5)
    
    # resample cbbox accordingly
    res_cbbox = [(round(cbbox[i].start*spacing[i]), 
                  round(cbbox[i].stop*spacing[i])) for i in range(3)]
    
    res_cmask = ndimage.zoom(cmask.astype(int), spacing)
    
    # center of cbbox
    res_cbbox0 = [round((res_cbbox[i][0]+res_cbbox[i][1])/2) for i in range(3)]
    
    # cmask is given realtive to cbbox, express relative to original volume
    g = np.zeros(res_data.shape[1:])
    g[res_cbbox[0][0]:res_cbbox[0][0]+res_cmask.shape[0], 
      res_cbbox[1][0]:res_cbbox[1][0]+res_cmask.shape[1],
      res_cbbox[2][0]:res_cbbox[2][0]+res_cmask.shape[2],] = res_cmask

    # extract volumes of dimension 2k x 2k x 2k
    k = int(h/2)
    slices = (
                slice(res_cbbox0[0]-k, res_cbbox0[0]+k),
                slice(res_cbbox0[1]-k, res_cbbox0[1]+k),
                slice(res_cbbox0[2]-k, res_cbbox0[2]+k)
             )

    crop = res_data[0][slices]

    g = torch.tensor(g)
    mask = g[slices]
    
    assert crop.shape == torch.Size([h,h,h])
    assert mask.shape == torch.Size([h,h,h])
    
    return median_malig, crop, mask

### extract nodule volumes, associated tumor masks and annotations

In [17]:
# labels: 1 -> malignant, 0 -> benign
labels = {}
new_id = 1
match = []
Path(f"{save_path}/crops").mkdir(parents=True, exist_ok=True)
Path(f"{save_path}/masks").mkdir(parents=True, exist_ok=True)

attributes = [
    "subtlety",
    "internalStructure",
    "calcification",
    "sphericity",
    "margin",
    "lobulation",
    "spiculation",
    "texture"
]

d = glob.glob(f"{base_path}/*")
d.sort()
# list of patient ids
pids = [i.split("/")[-1].split("-")[-1] for i in d]

# diameter has to be treated separately because of how the annotations are organized
avg_annotations = {}
for att in attributes:
    avg_annotations[att] = []
avg_annotations["diameter"] = []
avg_annotations["target"] = []
avg_annotations["path"] = []

for patient_id in tqdm(pids):
    #print(patient_id, new_id)
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == f"LIDC-IDRI-{patient_id}")[0]
    nodules = scan.cluster_annotations()
    if(len(nodules) == 0):
        continue
    k = 0
    for nodule in nodules:
        num_annotations = len(nodule)
        # only nodules with > 2 annotations are considered
        # if there are more than 4 annotations, clustering of annotations is ambiguous --> skip
        if(num_annotations > 4):
            print("skipping!")
        if(num_annotations > 2 and num_annotations <= 4):
            try:
                median_malig, crop, mask = process_nodule_vol(nodule, h=h)
                str_new_id = str(new_id).zfill(4)
                append = False
                if(median_malig > 3):
                    avg_annotations["target"].append(1)
                    append = True
                    new_id += 1
                elif(median_malig < 3):
                    avg_annotations["target"].append(0)
                    append = True
                    new_id += 1
                if(append):
                    avg_annotations["diameter"].append(np.mean([ann.diameter for ann in nodule]))
                    avg_annotations["path"].append(f"{str_new_id}.pt")
                    for att in attributes:
                        avg_annotations[att].append(np.mean([vars(ann)[att] for ann in nodule]))
                        
                    match.append([patient_id, k, new_id])
                    torch.save(crop.clone(), f"{save_path}/crops/{str_new_id}.pt")
                    torch.save(mask.clone(), f"{save_path}/masks/{str_new_id}.pt")
            # if creation of crop fails for any reason, skip to next nodule
            except Exception as e:
                print(e)
                continue
        k += 1
with open(f"{save_path}/match.pkl", "wb") as handle:
    pickle.dump(match, handle)
    
with open(f"{save_path}/annotations_new.pkl", "wb") as handle:
    pickle.dump(avg_annotations, handle)
os.remove("output.txt")

100%|██████████| 5/5 [00:16<00:00,  3.38s/it]


In [14]:
#create pandas dataframe from dictionary and save as pickle
with open(f"{save_path}/annotations_new.pkl", "rb") as f:
    avg_annotations = pickle.load(f)
df_ann = pd.DataFrame.from_dict(avg_annotations)
df_ann.to_pickle(f"{save_path}/annotations_df.pkl")